In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [11]:
# Load the dataset
lines = open('./dorothea_train.data').read().splitlines()

In [12]:
# Declare Empty X matrix
tmp=np.zeros(shape=(800,100000))
X=pd.DataFrame(tmp)

# Now fill X matrix
for i in range(0,len(lines)):
    row=lines[i]
    row=row.split()
    row = list(map(int, row)) ## convert string indexes as int indexes
    row=np.array(row) ## Convert to numpy array to apply arithmetic operation
    row=row-1 ## convert index from 1 to 100000 to 0 to 99999
    X.ix[i][row]=1

In [13]:
#Calculated Centred Matrix A
mean=X.mean()
A=X-mean
A=A.as_matrix()   ## Convert to numpy from DF
X=X.as_matrix()

In [14]:
#Read lables
lines = open('./dorothea_train.labels').read().splitlines()
train_lables=lines   

In [15]:
## Prepare train and test data
features=1000
data=X[:,:features] ## Select 1,000 features
train=data

df=pd.DataFrame(train)
tmp=pd.Series(train_lables)
df.loc[:,features]=tmp

#Now split as per lables
df0=df[df[features]=='-1'] 
df1=df[df[features]=='1']  
## Remove class lables now
del df0[features] ## DF containing class -1 
del df1[features] ## DF containing class +1
## convert to numpy arrays
class0=df0.as_matrix()
class1=df1.as_matrix()

In [16]:
########################################## LDA ###################################

In [17]:
m=data.mean(0)
m1=class0.mean(0)
m2=class1.mean(0)

In [18]:
## Within Class
A1=class0-m1
A2=class1-m2
S1 = np.dot(A1.T, A1)
S2 = np.dot(A2.T, A2)
SW = S1 + S2 + np.identity(len(S1[0]))
SW.shape

(1000, 1000)

In [20]:
# Between Class
SB = 0
M1 = m1.reshape(1000,1) - m.reshape(1000,1)
temp = np.dot(M1, M1.T)
temp = len(class0)*temp
SB+=temp

M2 = m2.reshape(1000,1) - m.reshape(1000,1)
temp = np.dot(M2, M2.T)
temp = len(class1)*temp
SB += temp

In [21]:
reduced_features=2
F = np.dot(np.linalg.inv(SW), SB)
eigen_values, eigen_vectors = np.linalg.eig(F)
## Sort
indexes = eigen_values.argsort()[::-1]
transformation = eigen_vectors[:, indexes[0:reduced_features]] ## Set num of features to choose

## Transform
Xlda = np.dot(train, transformation)

In [22]:
## Plot the scattered Points
import matplotlib.pyplot as plt
# plt.plot(Xlda, 'ro')
# plt.show()
plt.scatter(Xlda[:,0], Xlda[:,1])
plt.show()

/home/prakhar/Downloads/enter/lib/python3.5/site-packages/numpy/core/numeric.py:533: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order, subok=True)


In [23]:
############## READ TEST DATA ##########################
lines_test=open('./dorothea_valid.data').read().splitlines() ## Test
# Declare Empty X matrix
tmp=np.zeros(shape=(350,100000))
X_test=pd.DataFrame(tmp)

# Now fill X matrix
for i in range(0,len(lines_test)):
    row=lines_test[i]
    row=row.split()
    row = list(map(int, row)) ## convert string indexes as int indexes
    row=np.array(row) ## Convert to numpy array to apply arithmetic operation
    row=row-1 ## convert index from 1 to 100000 to 0 to 99999
    X_test.ix[i][row]=1

In [24]:
X_test=X_test.as_matrix()
X_test=X_test[:,:features] ## Select 1,000 features
# X_test=np.transpose(X_test)   ## X_test is now d*n

# Transform
X_test_trasformed = np.dot(X_test, transformation)
X_test_trasformed.shape

(350, 2)

In [25]:
####################### BAyes ##############################3

In [26]:
train=Xlda
test=X_test_trasformed

In [27]:
 #Read lables
lines_test = open('./dorothea_valid.labels').read().splitlines()
test_lables=lines_test

In [28]:
## Split train data according to classes
#Add class lables to data
df=pd.DataFrame(train)
tmp=pd.Series(train_lables)
df.loc[:,reduced_features+1]=tmp
#Now split as per lables
df0=df[df[reduced_features+1]=='-1'] 
df1=df[df[reduced_features+1]=='1']  

## Remove class lables now
del df0[reduced_features+1] ## DF containing class -1 
del df1[reduced_features+1] ## DF containing class +1
## convert to numpy arrays
class0=df0.as_matrix()
class1=df1.as_matrix()

In [29]:
## Now find mean and covariance
u1=class0.mean(0)
u2=class1.mean(0)
e1=np.cov(class0,rowvar=False)
e2=np.cov(class1,rowvar=False) 
## Also calculate posterior prob
class0_pos=len(class0)/len(train)
class1_pos=len(class1)/len(train)

In [30]:
co1=e1.diagonal()
co2=e2.diagonal()
co1=np.dot(np.transpose(co1),co1)
co2=np.dot(np.transpose(co2),co2)

In [31]:
## Calculate p(x) from Gaussian Formula
import math
count=0
for i in range (0 , len(test)):
    ## CLass 0
    sample=test[i]
    sm=0
    for j in range (0,len(sample)): 
        xminusu=(sample[j]-u1[j])
        xminusu=xminusu * xminusu
        tmp=xminusu/e1[j][j]
        tmp=tmp*(-0.5)
        e=math.exp(tmp)
        e=e/(math.sqrt(2*math.pi*e1[j][j]))
        sm=sm+e
    p0=sm*class0_pos
    
    ## Class 1
    sample=test[i]
    sm=0
    for j in range (0,len(sample)): 
        xminusu=(sample[j]-u2[j])
        xminusu=xminusu * xminusu
        tmp=xminusu/e2[j][j]
        tmp=tmp*(-0.5)
        e=math.exp(tmp)
        e=e/(math.sqrt(2*math.pi*e2[j][j]))
        sm=sm+e
    p1=sm*class1_pos
    
    
    if(p0>p1):
        result='-1'
    else : 
        result='1'
    if(result==test_lables[i]):
        count+=1
    

print(float(count)/len(test))

0.8971428571428571


/home/prakhar/Downloads/enter/lib/python3.5/site-packages/ipykernel/__main__.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
/home/prakhar/Downloads/enter/lib/python3.5/site-packages/ipykernel/__main__.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
/home/prakhar/Downloads/enter/lib/python3.5/site-packages/ipykernel/__main__.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
/home/prakhar/Downloads/enter/lib/python3.5/site-packages/ipykernel/__main__.py:27: ComplexWarning: Casting complex values to real discards the imaginary part


In [32]:
from sklearn.naive_bayes import GaussianNB
abc=GaussianNB()
abc.fit(train,train_lables)
print(abc.score(test,test_lables))

0.888571428571


/home/prakhar/Downloads/enter/lib/python3.5/site-packages/sklearn/naive_bayes.py:378: ComplexWarning: Casting complex values to real discards the imaginary part
  self.theta_[i, :] = new_theta
